# Links

- [Liu 2025: Cell type–specific 3D-genome organization and transcription regulation in the brain](https://www.science.org/doi/10.1126/sciadv.adv2067)
- [GitHub repo for Liu 2025](https://github.com/ZhuangLab/Chromatin_Analysis_MOp/)

# Processing FOF-CT files

## Docs

- [Docs: 4DN FISH Omics Format - Chromatin Tracing (FOF-CT)](https://fish-omics-format.readthedocs.io/en/latest/)
- [DNA-Spot/Trace Data core table](https://fish-omics-format.readthedocs.io/en/latest/core.html)
- [Cell Data table](https://fish-omics-format.readthedocs.io/en/latest/cell.html)

Note: FOV = 'fields of view'

## Examining files

### Examining DNA-Spot/Trace Data core table

In [16]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"
cd "$sample_dir/biorep1_techrep1"

# # ==== Examine header info

# Number of header lines
echo $(grep '^["#]' dna_spot_trace_core.csv | wc -l)" header lines, including col labels"
echo $(grep ',,,' dna_spot_trace_core.csv | wc -l)" header lines, excluding col labels"

# Column labels
echo -e "\nColumn labels:"
grep -v ',,,' dna_spot_trace_core.csv | head -n 1 | sed 's/.*(//;s/).*//' | tr ',' '\n' | awk 'BEGIN{c=1}{print c "\t" $0; c += 1}'

# Column label desc
echo -e "\nColumn label desc:"
grep ',,,' dna_spot_trace_core.csv | tail -n 7 | sed 's/,,*//g'

23 header lines, including col labels
22 header lines, excluding col labels

Column labels:
1	Spot_ID
2	Trace_ID
3	X
4	Y
5	Z
6	Chrom
7	Chrom_Start
8	Chrom_End
9	Chrom_order
10	Cell_ID
11	FOV_ID
12	CellID_byFOV
13	RNA_experiment_ID
14	DNA_experiment_ID
15	Sample_ID

Column label desc:
##XYZ_unit=nm
#Spot_ID:unique DNA spot identifier across 4dn_FOF-CT_core and 4dn_FOF-CT_demultiplxeing for the same replicate
#Trace_ID:number format as fovXX_cellXX_chrXX_chromatidXX
#Chrom_order:order of DNA region on its respective chromosome
#Cell_ID:unique cell identifier across the datatables
#FOV:FOV ID across datatables for the same replicate
#CellID_byFOVspecific cell identifier relative to the FOV


In [9]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"
cd "$sample_dir/biorep1_techrep1"

# ==== Check uniqueness of cells
echo -e "Uniqueness of cells"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | cut -d ',' -f 10-12 | sort | uniq | wc -l)" unique combos of 'Cell_ID, FOV_ID, CellID_byFOV'"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | cut -d ',' -f 10 | sort | uniq | wc -l)" unique 'Cell_ID'"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | cut -d ',' -f 11-12 | sort | uniq | wc -l)" unique combos of 'FOV_ID, CellID_byFOV'"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | cut -d ',' -f 12 | sort | uniq | wc -l)" unique 'CellID_byFOV'"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | cut -d ',' -f 11 | sort | uniq | wc -l)" unique 'FOV_ID'"


# ==== Check uniqueness of rows
echo -e "\nUniqueness of rows"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | wc -l)" unique 'rows'"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | cut -d ',' -f 1 | sort | uniq | wc -l)" unique 'Spot_ID'"
echo $(grep -v '^["#]' dna_spot_trace_core.csv | cut -d ',' -f 2,9 | sort | uniq | wc -l)" unique combos of 'Trace_ID, Chrom_order'"


Uniqueness of cells
12725 unique combos of 'Cell_ID, FOV_ID, CellID_byFOV'
12725 unique 'Cell_ID'
12725 unique combos of 'FOV_ID, CellID_byFOV'
253 unique 'CellID_byFOV'
148 unique 'FOV_ID'

Uniqueness of rows
8258329 unique 'rows'
8258329 unique 'Spot_ID'
8258329 unique combos of 'Trace_ID, Chrom_order'


In [11]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"
cd "$sample_dir/biorep1_techrep1"

# ==== Check that the chromosome component of 'Trace_ID' (fovXX_cellXX_chrXX_chromatidXX) corresponds to the 'Chrom' column
ndiff=$(awk -F '[,_]' '{ if ($1 !~ /^[#"]/){ gsub(/chr/, "", $9); if ($4 != $9) { print $4 "\t" $9 } } }' dna_spot_trace_core.csv | wc -l)
if [[ $ndiff -eq 0 ]]; then
    echo "✓ Chromosome component of 'Trace_ID' (fovXX_cellXX_chrXX_chromatidXX) corresponds to the 'Chrom' column"
else
    echo "✗ Chromosome component of 'Trace_ID' (fovXX_cellXX_chrXX_chromatidXX) does NOT correspond to the 'Chrom' column for $ndiff lines"
fi


# ==== Examine size of each region
echo -e "\nSize of each region: (1st 10 lines)"
awk -F ',' '{ if ($1 !~ /^[#"]/){ print $8 - $7 } }' dna_spot_trace_core.csv | head

✓ Chromosome component of 'Trace_ID' (fovXX_cellXX_chrXX_chromatidXX) corresponds to the 'Chrom' column

Size of each region: (1st 10 lines)
8595
8623
11242
10504
8112
9570
8405
7269
8405
7187


### Examining Cell Data table

In [55]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"
cd "$sample_dir/biorep1_techrep1"

# # ==== Examine header info

# Number of header lines
echo $(grep '^["#]' cell_data.csv | wc -l)" header lines, including col labels"
echo $(grep ',,,' cell_data.csv | wc -l)" header lines, excluding col labels"

# Column labels
ncols_gene=$( grep -v ',,,' cell_data.csv | head -n 1 | sed 's/.*neuron_identity,//' | tr ',' '\n' | wc -l )
echo -e "\nColumn labels (excluding $ncols_gene gene xp columns):"
grep -v ',,,' cell_data.csv | head -n 1 | sed 's/neuron_identity,.*/neuron_identity/;s/.*(//;s/).*//' | tr ',' '\n' | awk 'BEGIN{c=1}{print c "\t" $0; c += 1}'

# Column label desc
echo -e "\nColumn label desc:"
grep ',,,' cell_data.csv | tail -n 7 | sed 's/,,*//g'

23 header lines, including col labels
22 header lines, excluding col labels

Column labels (excluding 242 gene xp columns):
1	Cell_ID
2	FOV_ID
3	cell_volume_from_merlin
4	cell_center_x_global
5	cell_center_y_global
6	RNA_experiment_ID
7	Sample_ID
8	cluster_subclass
9	cluster_class
10	neuron_identity

Column label desc:
##XYZ_unit=micron
#Cell_ID:unique cell identifier across the datatables
#cell_center_x_global:x coordinate of the cell relative to all cells for the experiment replicate
#cell_center_y_global:y coordinate of the cell relative to all cells for the experiment replicate
#cluster_subclass:transcriptionally defined cell cluster lable at subclass level
#cluster_class:transcriptionally defined cell cluster lable at a higer level
"#RNA count for gene A:decoded raw RNA count for gene A (e.g. gene 1700022I11Rik)"


In [116]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"
biorep="$sample_dir/biorep1_techrep1"

# # ==== Count cells & clusters

# Number of cells
echo -e $(grep -v '^["#]' "$biorep/cell_data.csv" | wc -l)" cells (lines)"

# Number of clusters
nclust_sub=$( grep -v '^["#]' "$biorep/cell_data.csv" | cut -d ',' -f 8 | sort | sed 's/ //g' | uniq -c | awk '{ print $1 "\t" $2 }' | sort -k1,1nr )
echo -e "\n"$(echo -e "$nclust_sub" | wc -l)" clusters (only via 'cluster_subclass')"
nclust=$( grep -v '^["#]' "$biorep/cell_data.csv" | cut -d ',' -f 8-10 | sort | sed 's/ //g' | uniq -c | awk '{ print $1 "\t" $2 }' | sort -k1,1nr )
echo -e $(echo -e "$nclust" | wc -l)" clusters (via combo of 'cluster_subclass, cluster_class, neuron_identity'):"
echo -e "$nclust"

# Number of clusters (only via cluster_class)
nclust_class=$( grep -v '^["#]' "$biorep/cell_data.csv" | cut -d ',' -f 9 | sort | sed 's/ //g' | uniq -c | awk '{ print $1 "\t" $2 }' | sort -k1,1nr )
echo -e "\n"$(echo -e "$nclust_class" | wc -l)" clusters (only via 'cluster_class')"
echo -e "$nclust_class"

17856 cells (lines)

22 clusters (only via 'cluster_subclass')
22 clusters (via combo of 'cluster_subclass, cluster_class, neuron_identity'):
2184	L2/3IT,Gluta,Neuronal
1803	Astro,Astro,Non-Neuronal
1741	L6CT,Gluta,Neuronal
1474	L5IT,Gluta,Neuronal
1294	other,other,other
1088	L5ET,Gluta,Neuronal
1039	Endo,Endo,Non-Neuronal
1029	L4/5IT,Gluta,Neuronal
1029	OPC,Oligo,Non-Neuronal
972	Oligo,Oligo,Non-Neuronal
789	L6IT,Gluta,Neuronal
711	Micro,Micro,Non-Neuronal
549	Pvalb,GABA,Neuronal
391	L5/6NP,Gluta,Neuronal
380	Sst,GABA,Neuronal
274	VLMC,VLMC,Non-Neuronal
266	Peri,Peri,Non-Neuronal
205	Vip,GABA,Neuronal
198	SMC,SMC,Non-Neuronal
185	L6b,Gluta,Neuronal
180	Lamp5,GABA,Neuronal
75	Sncg,GABA,Neuronal

10 clusters (only via 'cluster_class')
8881	Gluta
2001	Oligo
1803	Astro
1389	GABA
1294	other
1039	Endo
711	Micro
274	VLMC
266	Peri
198	SMC


## Extracting relevant information

In [211]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"

# ==== Reformat relevant columns from 'cell_data.csv':
# 1 	Cell_ID
# 8 	cluster_subclass
# 9 	cluster_class
# 10	neuron_identity

# # Testing on one biorep
# biorep="$sample_dir/biorep1_techrep1"
# rep_id=$(basename "$biorep" | sed 's/_/./;s/[A-z]*//g' )
# awk -F ',' -v rep_id="$rep_id" 'BEGIN{OFS=","}{ if ($(NF-1) != ""){ 
#     if ($1 ~ /Cell_ID/){ print "#rep_id", "#" tolower($10), "#" tolower($9), "#" tolower($8), "#cell_id" }
# 	  else { gsub(/ /, "_", $8); print rep_id, tolower($10), tolower($9), tolower($8), $1 } } }' "$biorep/cell_data.csv" | sort -t ',' -k2,5 | head  | tr ',' '\t'


for biorep in $( find "$sample_dir" -type d -name 'biorep*_techrep*' ); do
	if [ -f "$biorep/cells.csv" ]; then
		continue
	fi
    basename "$biorep"
	if [ ! -f "$biorep/cell_data.csv" ] & [ -f "$biorep/cell_data.csv.gz" ]; then
		gunzip "$biorep/cell_data.csv.gz"
	fi

    rep_id=$(basename "$biorep" | sed 's/_/./;s/[A-z]*//g' )
    awk -F ',' -v rep_id="$rep_id" 'BEGIN{OFS=","}{ if ($(NF-1) != ""){ 
        if ($1 ~ /Cell_ID/){ print "#rep_id", "#" tolower($10), "#" tolower($9), "#" tolower($8), "#cell_id" }
        else { gsub(/ /, "_", $8); print rep_id, tolower($10), tolower($9), tolower($8), $1 } } }' "$biorep/cell_data.csv" | sort -t ',' -k2,5 > "$biorep/cells.csv"

	gzip "$biorep/cell_data.csv"
done

biorep3_techrep1
biorep1_techrep1
biorep2_techrep1
biorep3_techrep2


In [222]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"

# ==== Reformat relevant columns from 'dna_spot_trace_core.csv':
# 2 	Trace_ID  (fovXX_cellXX_chrXX_chromatidXX)
# 3 	X
# 4 	Y
# 5 	Z
# 7 	Chrom_Start
# 8 	Chrom_End
# 9 	Chrom_order
# 10	Cell_ID

# # Testing on one biorep
# biorep="$sample_dir/biorep1_techrep1"
# rep_id=$(basename "$biorep" | sed 's/_/./;s/[A-z]*//g' )
# awk -F ',' -v rep_id="$rep_id" 'BEGIN{OFS=","}{ if ($(NF-1) != ""){ 
# 	tmp=$9","$7","$8","$3","$4","$5;
# 	if ($2 == "Trace_ID"){ molecule="#rep_id,#cell_id,#chrom,#hmlg"; tmp="#"gensub(/,/, ",#", "g", tolower(tmp)) }
# 	else { molecule=rep_id","$10","gensub(/.+_.+_(.+)_(.+)/, "\\1,\\2", "g", $2) };
# 	print molecule, tmp } }' "$biorep/dna_spot_trace_core.csv" | sort -t ',' -k2,2 -k3,3n -k4,4n -k5,5n | head | tr ',' '\t' | sed -r 's/(\.[0-9])[0-9]*/\1/g'


for biorep in $( find "$sample_dir" -type d -name 'biorep*_techrep*' ); do
	if [ -f "$biorep/dna_merfish.csv" ]; then
		continue
	fi
    basename "$biorep"
	if [ ! -f "$biorep/dna_spot_trace_core.csv" ] & [ -f "$biorep/dna_spot_trace_core.csv.gz" ]; then
		gunzip "$biorep/dna_spot_trace_core.csv.gz"
	fi

    rep_id=$(basename "$biorep" | sed 's/_/./;s/[A-z]*//g' )
    awk -F ',' -v rep_id="$rep_id" 'BEGIN{OFS=","}{ if ($(NF-1) != ""){ 
    	tmp=$9","$7","$8","$3","$4","$5;
    	if ($2 == "Trace_ID"){ molecule="#rep_id,#cell_id,#chrom,#hmlg"; tmp="#"gensub(/,/, ",#", "g", tolower(tmp)) }
    	else { molecule=rep_id","$10","gensub(/.+_.+_(.+)_(.+)/, "\\1,\\2", "g", $2) };
    	print molecule, tmp } }' "$biorep/dna_spot_trace_core.csv" | sort -t ',' -k2,2 -k3,3n -k4,4n -k5,5n > "$biorep/dna_merfish.csv"

	gzip "$biorep/dna_spot_trace_core.csv"
done

## Consolidating data across bioreps (excluding secondary tech reps per biorep)

In [223]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"

allreps_dir="$sample_dir/all_techrep1"
cells_file="$allreps_dir/cells.csv"

# ==== Consolidate cells across bioreps (1st techrep only)

mkdir -p "$allreps_dir"
head -n 1 $(find "$sample_dir" -type f -path '*/biorep*_techrep1/cells.csv' | head -n 1) > "$cells_file" # Write header
grep -r "$sample_dir/"biorep*_techrep1/cells.csv -hve '^#' | sort -t ',' -k1,5 >> "$cells_file" # Write cells

In [224]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"

allreps_dir="$sample_dir/all_techrep1"
cells_file="$allreps_dir/cells.csv"

# ==== Count cells & clusters across all bioreps

# Number of cells
echo -e $(grep -v '^["#]' "$cells_file" | wc -l)" cells (lines)"

# Number of clusters
nclust=$( grep -v '^["#]' "$cells_file" | cut -d ',' -f 4 | sort | sed 's/ //g' | uniq -c | awk '{ print $1 "\t" $2 }' | sort -k1,1nr )
echo -e "\n"$(echo -e "$nclust" | wc -l)" clusters (only via 'cluster_subclass')"
nclust_trio=$( grep -v '^["#]' "$cells_file" | cut -d ',' -f 2-4 | sort | sed 's/ //g' | uniq -c | awk '{ print $1 "\t" $2 }' | sort -k1,1nr )
echo -e $(echo -e "$nclust_trio" | wc -l)" clusters (via combo of 'cluster_subclass, cluster_class, neuron_identity'):"
echo -e "$nclust_trio"

echo -e "\nNumber of (subclass-based) clusters with >= [cutoff] cells (besides 'other')"
for cutoff in 1000 2000; do
    echo -e $( echo -e "$nclust" | awk -vcutoff="$cutoff" '{if (($1 >= cutoff) && ($2 != "other")) {print $0}}' | wc -l )" clusters with >= $cutoff cells"
done

# Number of clusters (only via cluster_class)
nclust_class=$( grep -v '^["#]' "$cells_file" | cut -d ',' -f 3 | sort | sed 's/ //g' | uniq -c | awk '{ print $1 "\t" $2 }' | sort -k1,1nr )
echo -e "\n"$(echo -e "$nclust_class" | wc -l)" clusters (only via 'cluster_class')"
echo -e "$nclust_class"

47999 cells (lines)

22 clusters (only via 'cluster_subclass')
22 clusters (via combo of 'cluster_subclass, cluster_class, neuron_identity'):
5433	non-neuronal,oligo,oligo
5348	neuronal,gluta,l6_ct
4833	non-neuronal,astro,astro
4721	neuronal,gluta,l2/3_it
3507	non-neuronal,endo,endo
2989	neuronal,gluta,l5_it
2945	neuronal,gluta,l4/5_it
2520	other,other,other
2366	neuronal,gluta,l6_it
2150	neuronal,gluta,l5_et
1953	non-neuronal,oligo,opc
1745	non-neuronal,micro,micro
1367	neuronal,gaba,pvalb
1001	neuronal,gaba,sst
955	neuronal,gluta,l6b
891	non-neuronal,peri,peri
791	neuronal,gluta,l5/6_np
716	non-neuronal,vlmc,vlmc
571	non-neuronal,smc,smc
500	neuronal,gaba,vip
496	neuronal,gaba,lamp5
201	neuronal,gaba,sncg

Number of (subclass-based) clusters with >= [cutoff] cells (besides 'other')
13 clusters with >= 1000 cells
9 clusters with >= 2000 cells

10 clusters (only via 'cluster_class')
22265	gluta
7386	oligo
4833	astro
3565	gaba
3507	endo
2520	other
1745	micro
891	peri
716	vlmc
571	smc


In [226]:
%%bash

sample_dir="$HOME/noble_lab/repos/Chromatin_Analysis_MOp/data/MOp_wt"
ncells_cutoff=2000

allreps_dir="$sample_dir/all_techrep1"
cells_file="$allreps_dir/cells.csv"

# ==== Get cells belonging to 'top' clusters (clusters with >= cutoff cells)
clusters_all=$( grep -v '^["#]' "$cells_file" | cut -d ',' -f 4 | sort | sed 's/ //g' | uniq -c | awk '{ print $1 "\t" $2 }' | sort -k1,1nr )
clusters_top=$( echo -e "$clusters_all" | awk -vcutoff="$ncells_cutoff" '{if (($1 >= cutoff) && ($2 != "other")) {print $2}}' )
nclust_top=$( echo -e "$clusters_top" | wc -l )

clusters_top_re="(^"$( echo -e "$clusters_top" | tr '\n' '|' | sed 's/|$//;s/|/$|^/g;s:/:.:g' )"$)"
cells_topclust_file="$allreps_dir/cells.top${nclust_top}clust.csv"
    
head -n 1 "$cells_file" | cut -d ',' -f 1,4,5 > "$cells_topclust_file"
awk -F',' 'BEGIN{OFS=","}{ if ($4 ~ /'"$clusters_top_re"'/){ print $1, $4, $5 } }' "$cells_file"  >> "$cells_topclust_file"


# ==== Label DNA MERFISH data from cells belonging to 'top' clusters (clusters with >= cutoff cells); discard data for other cells

dna_allreps_file="$allreps_dir/dna_merfish.top${nclust_top}clust.csv"

# FIXME put header in dna_allreps_file
    
# for biorep in $( find "$sample_dir" -type d -name 'biorep*_techrep1' ); do

biorep="$sample_dir/biorep1_techrep1"
rep_id=$(basename "$biorep" | sed 's/_/./;s/[A-z]*//g' )


# awk -F ',' -v rep_id="$rep_id" '{ if (($1 == rep_id) || ($1 ~ /rep_id/)){ print $2 "," $3 } }' "$cells_topclust_file" | sort -t ',' -k 2b,2 | head; echo; echo
    
awk -F ',' -v rep_id="$rep_id" '{ if (($1 == rep_id) || ($1 ~ /rep_id/)){ print $2 "," $3 } }' "$cells_topclust_file" | sort -t ',' -k 2b,2 | \
    join -t ',' -j 2 --header --check-order "$biorep/dna_merfish.csv" - | grep -v '^#' >> "$dna_allreps_file"


# ==== Segregate DNA MERFISH data by cluster

# # Make directories per cluster, write header for sorted dna_merfish file
# for cluster in $clusters_top; do
#     mkdir -p "$allreps_dir/$cluster"
#     head -n 1 $(find "$sample_dir" -type f -path '*/biorep*_techrep1/dna_merfish.csv' | head -n 1) > "$allreps_dir/$cluster/dna_merfish.$cluster.csv"
# done

# awk -F',' -v outdir="$allreps_dir" 'BEGIN{OFS=","}{ if ($1 !~ /^#/) \
#     { print $1, $2, $3, $4, $5, $6, $7, $8, $9, $10 > outdir"/"$11"/dna_merfish."$11".csv" } }' "$dna_allreps_file"

#cell_id,#rep_id,#chrom,#hmlg,#chrom_order,#chrom_start,#chrom_end,#x,#y,#z,#cluster_subclass
100066287421962522127773928555604620338,1.1,1,1,0,3742742,3759944,199.11620879999998,181.73214,6.241104125,oligo
100066287421962522127773928555604620338,1.1,1,1,1,6245958,6258969,198.89543519999998,181.7166636,5.747276583333334,oligo
100066287421962522127773928555604620338,1.1,1,1,3,9627926,9637875,198.8128656,182.5055964,5.478145583333334,oligo
100066287421962522127773928555604620338,1.1,1,1,5,11247744,11257616,198.67533479999992,181.41873119999997,6.244514166666666,oligo
100066287421962522127773928555604620338,1.1,1,1,6,13741888,13757922,198.06651720000002,181.65839039999997,6.576315416666667,oligo
100066287421962522127773928555604620338,1.1,1,1,12,21732182,21745770,197.86926600000004,181.2541572,5.695393,oligo
100066287421962522127773928555604620338,1.1,1,1,14,23749258,23759965,197.374518,181.1868192,6.073499333333333,oligo
100066287421962522127773928555604620338,1.1,1,1,15,23956510,2397023